In [55]:
from __future__ import print_function
import pandas as pd
import numpy as np
import multiprocessing as mp
import os
from sklearn.externals import joblib
import sys
sys.path.append("../../../ddt/")
from utility import FeatureGenerator
from glob import glob
from sklearn.externals import joblib
import pickle
import pandas as pd
import numpy as np

In [12]:
models_info = pd.read_csv("../../all_models_info.csv")
models_info.head()

,name,f1,roc_auc,model_type,fam,version
0,15_hydroxyprostaglandin_dehydrogenase_nad_,1.0,1.0,RandomForestClassifier,Other,py27
1,1_25_dihydroxyvitamin_d_3_24_hydroxylase_mitoc...,1.0,1.0,XGBClassifier,Other,py27
2,1_acyl_sn_glycerol_3_phosphate_acyltransferase...,1.0,1.0,XGBClassifier,Enzyme,py27
3,2_5a_dependent_ribonuclease,1.0,1.0,XGBClassifier,Kinase,py35
4,2_acylglycerol_o_acyltransferase_2,1.0,1.0,SVC,Enzyme,py35


In [48]:
py_version = "27" if sys.version_info[0] < 3 else "35"
zinc_dir = os.path.abspath("../../../ZINC_sdf")
sdf_files = glob(os.path.join(zinc_dir, "*/*/*.sdf"))
print("SDF FILE COUNT: {}".format(len(sdf_files)))
models_dir = os.path.abspath("../../models")

SDF FILE COUNT: 19185


In [57]:
def get_models():
    for model_name, model_type in models_info[models_info['version'] == 'py35'][['name', 'model_type']].values:
        model_file = os.path.join(models_dir, model_name)
        if model_type == "RandomForestClassifier": model = joblib.load(model_file + '.rfc')
        elif model_type == "SVC": model = joblib.load(model_file + '.svc')
        elif model_type == "MLPClassifier": model = joblib.load(model_file + '.mlp')  
        elif model_type == 'XGBClassifier': model = pickle.load(open(model_file + '.xgb', 'rb'))
        yield model_name, model_type, model

In [45]:
results = {}

In [ ]:
def get_zinc_id(_file):
    zinc_ids = []
    content = None
    with open(_file, 'r') as f:
        content = f.readlines()
    if content is None: return None
    content = content.split("/n")
    zinc_ids.append(content[0]) 

In [ ]:
ft = FeatureGenerator()
for _name, _type, _model in get_models():
    for sdf in sdf_files:
        _, sdf_filename = os.path.split(sdf)
        ft.load_sdf(sdf)
        features = ft.extract_tpatf()
        features = np.array(features).reshape((-1, 2692))
        print(features.shape)
        results[_name] = {}
        results['name']['type'] = _type
        #results['name'][sdf_filename] = 
        break
    break

/home/mhassan/.conda/envs/ml/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
